# **IMPLEMENTACIÓN VGG 16**

**Se usarán los mejores modelos para predecir aún mejor los accidentes**

- VGG
- Mobile 
- Transfer Learning

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from keras import layers

Using TensorFlow backend.


In [0]:

from os import scandir, getcwd
from os.path import abspath

#Función para listar archivos 
def listarArchivos(ruta = getcwd()):
    return [abspath(arch.path) for arch in scandir(ruta) if arch.is_file()]

#Funcion listar los frames de los videos
def listarImagen(direccion):
  im_ = tf.keras.preprocessing.image.load_img((direccion), target_size=(224, 224))
  ima_ = tf.keras.preprocessing.image.img_to_array(im_)
  return ima_


#--------- Conjunto de entrenamiento para accidente y no accidente
x_train_Accidente = [ listarImagen(i) 
                      for i in listarArchivos('/content/drive/My Drive/Colab Notebooks/Proyecto/xEntrenamientoAccidente/img/')]

x_train_No_Accidente = [ listarImagen(i)
                        for i in listarArchivos('/content/drive/My Drive/Colab Notebooks/Proyecto/xEntrenamientoNoAccidente/img/')]


# ---------- Conjunto de prueba
x_test_Accidente = [ listarImagen(i) 
                      for i in listarArchivos('/content/drive/My Drive/Colab Notebooks/Proyecto/xPruebaAccidente/img/')]

x_test_No_Accidente = [ listarImagen(i) 
                        for i in listarArchivos('/content/drive/My Drive/Colab Notebooks/Proyecto/xPruebaNoAccidente/img/')]

In [3]:
# Generando las etiquetas para cada conjunto de datos

y_train_Accidente = [1 for _ in range(len(x_train_Accidente))]
y_train_NoAccidente = [ 0 for _ in range(len(x_train_No_Accidente)) ]

y_test_Accidente = [1 for _ in range(len(x_test_Accidente))]
y_test_NoAccidente = [0 for _ in range(len(x_test_No_Accidente))]


print (len(y_train_Accidente), len(y_train_NoAccidente), len(y_test_Accidente), len(y_test_NoAccidente))
print ( len(x_train_Accidente), len(x_train_No_Accidente), len(x_test_Accidente), len(x_test_No_Accidente))

1042 1042 1084 1084
1042 1042 1084 1084


In [4]:
# Concatenando los X de entrenamiento y prueba para generar un mismo conjunto de datos 
x_train_Accidente = np.array(x_train_Accidente)
x_train_NoAccidente = np.array(x_train_No_Accidente)

x_test_Accidente = np.array(x_test_Accidente)
x_test_NoAccidente = np.array(x_test_No_Accidente)

y_train_Accidente = np.array(y_train_Accidente)
y_train_NoAccidente = np.array(y_train_NoAccidente)

y_test_Accidente = np.array(y_test_Accidente)
y_test_NoAccidente = np.array(y_test_NoAccidente)

# -----------------------------

X_train = np.concatenate( (x_train_NoAccidente, x_train_Accidente), axis = 0 )
X_test = np.concatenate ( (x_test_NoAccidente, x_test_Accidente), axis = 0 )

Y_train = np.concatenate (  (y_train_NoAccidente, y_train_Accidente), axis = 0 )
Y_test = np.concatenate ( (y_test_NoAccidente, y_test_Accidente), axis = 0 )


print (X_train.shape, Y_train.shape, X_test.shape , Y_test.shape)

(2084, 224, 224, 3) (2084,) (2168, 224, 224, 3) (2168,)


In [0]:
X_train = X_train/255
X_test = X_test/255

In [0]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(X_train, Y_train, random_state = 0)
X_test, Y_test = shuffle (X_test, Y_test, random_state = 0)

In [0]:
# -------------------------------------------------- RESNET ---------------------------------------------
modeoResnet = tf.keras.applications.ResNet101V2(weights = "imagenet")

In [79]:
# ------------------------ CONGELAR TODAS LAS CAPAS ------------------------------

# --------------------------- REMOVIENDO ÚLTIMA CAPA DENSA DE 1000 neuronas por 3 capas densas y la última de 2 neuronas  --------------------------- 
for capa in modeoResnet.layers:
  capa.trainable = False

x = modeoResnet.layers[-2].output

#Agregando capas densas
x = tf.keras.layers.Dense(1024, activation= tf.keras.layers.LeakyReLU(alpha = 0.3))(x)
x = tf.keras.layers.Dense(512, activation= tf.keras.layers.LeakyReLU(alpha = 0.3))(x)
x = tf.keras.layers.Dense(256, activation= tf.keras.layers.LeakyReLU(alpha = 0.3))(x)
x = tf.keras.layers.Dense(2, activation= 'softmax')(x)


#Generando el modelo de transfer learning
modelo_transfer_Resnet = tf.keras.Model(modeoResnet.input, x)

for capa in modeoResnet.layers[:10]:
  capa.trainable = True

modelo_transfer_Resnet.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
opt = tf.keras.optimizers.Adam(lr=0.0025)
modelo_transfer_Resnet.compile(optimizer=opt, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
historyResnet = modelo_transfer_Resnet.fit(X_train, Y_train, epochs=16, batch_size=16,
                                       validation_data=(X_test, Y_test))

Epoch 1/16
131/131 [==============================] - 88s 669ms/step - loss: 1.2898 - accuracy: 0.8719 - val_loss: 1.7305 - val_accuracy: 0.6052
Epoch 2/16
131/131 [==============================] - 86s 655ms/step - loss: 0.0275 - accuracy: 0.9947 - val_loss: 2.0742 - val_accuracy: 0.5641
Epoch 3/16
131/131 [==============================] - 86s 655ms/step - loss: 0.0013 - accuracy: 0.9995 - val_loss: 3.6278 - val_accuracy: 0.5268
Epoch 4/16
131/131 [==============================] - 86s 656ms/step - loss: 7.5106e-04 - accuracy: 1.0000 - val_loss: 4.8982 - val_accuracy: 0.5341
Epoch 5/16
131/131 [==============================] - 86s 656ms/step - loss: 1.8712e-04 - accuracy: 1.0000 - val_loss: 4.1587 - val_accuracy: 0.4991
Epoch 6/16
131/131 [==============================] - 86s 655ms/step - loss: 2.6882e-05 - accuracy: 1.0000 - val_loss: 4.1755 - val_accuracy: 0.5042
Epoch 7/16
131/131 [==============================] - 86s 656ms/step - loss: 6.5444e-06 - accuracy: 1.0000 - val_loss:

In [0]:
plt.plot(historyResnet.history['accuracy'], label='Train Accuracy')
plt.plot(historyResnet.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [0]:
from tensorflow.keras.models import load_model
import keras.activations


#a = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/Proyecto/modeloResnetPesos.h5',
#                          custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU(alpha = 0.3)})

In [0]:
# ------------------------------------- FUNCIÓN VIDEO A ARREGLO IMÁGENES ----------------------------

import cv2 as cv
import matplotlib.pyplot as plt


def videoLista (direccion):
  
  video = cv.VideoCapture(direccion)
  listaFrames = [ tf.keras.preprocessing.image.img_to_array(cv.resize(cv.cvtColor(video.read()[1], cv.COLOR_BGR2RGB) , (224, 224)) )
             for _ in range(2,int(video.get(cv.CAP_PROP_FRAME_COUNT)))]
  return listaFrames



In [0]:
direccion = "/content/drive/My Drive/Colab Notebooks/Proyecto/VideoPruebaRender.mp4"
listaVideoAccidente = videoLista(direccion)

listaVideoAccidente = np.array(listaVideoAccidente)

In [78]:
listaPredicha = modelo_transfer_Resnet.predict(listaVideoAccidente)
listaPredicha

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [76]:
def predichoAHumano(listaPredicha):
  return listaPredicha.astype(int)

listaP = predichoAHumano(listaPredicha)

listaP

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1,

In [0]:
# ------------------- CREAR EL NUEVO VIDEO CON LAS ETIQUETAS PREDICHAS -------------------------
b = videoLista(direccion)
b = np.array(b).astype(int)


In [0]:
plt.figure(figsize=(5,5))

estado = False
for i in range(len(b)):
  if( (listaP[i][0] == 1) and (listaP[i][1] == 0) and estado == False):
    plt.imshow(b[i])
    plt.title("No Accidente", fontsize=15 ,color="black")
    plt.savefig("/content/drive/My Drive/Colab Notebooks/Proyecto/imagenesPredichas/v1/f" + str(i) + ".png")
  elif( (listaP[i][0] == 0) and (listaP[i][1] == 1) or estado == True):
    plt.imshow(b[i])
    plt.title("Accidente",fontsize=20 ,color="red")
    plt.savefig("/content/drive/My Drive/Colab Notebooks/Proyecto/imagenesPredichas/v1/f" + str(i) + ".png")
    estado = True
    
  